In [22]:
addprocs(7);

In [23]:
@everywhere begin
    using NetworkDiscovery
    using POMDPs
    using POMCP
    using POMDPToolbox
end

In [24]:
nodes = 100
comms = 5
probes = 30
p_inter = 0.01
p_intra = 0.3
N = 10

10

In [25]:
function est_rew(policy, nodes, comms, probes, p_intra, p_inter, N)
    sum = @parallel (+) for i in 1:N
        prob_rng = MersenneTwister(i)
        sim_rng = MersenneTwister(i)
        nw = generate_network(prob_rng, nodes, comms, p_intra, p_inter)
        pomdp = generate_problem(prob_rng, nw, probes, 1, 100.0, 10, 10, p_intra, p_inter)
        if isa(policy, POMCPSolver)
            policy = solve(policy, pomdp)
        end
        revealed = initial_belief(pomdp)
        sim = RolloutSimulator(rng=sim_rng, initial_state=nw, initial_belief=revealed)
        simulate(sim, pomdp, policy)
    end
end

est_rew (generic function with 1 method)

In [26]:
guess_rng = MersenneTwister(1)
policy = DiscoveryHeuristic(ProbeHighestDegree(true), GuessBasedOnNeighbors(guess_rng))

DiscoveryHeuristic(ProbeHighestDegree(true),GuessBasedOnNeighbors(MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,0]),Uint32[0x00000001])))

In [27]:
@time est_rew(policy, nodes, comms, probes, p_intra, p_inter, N)

elapsed time: 5.836341201 seconds (17208772 bytes allocated)


700.0

In [28]:
rollout_rng = MersenneTwister(1)
pomcp_rng = MersenneTwister(1)
rollout_policy = DiscoveryHeuristic(ProbeHighestDegree(true), GuessBasedOnNeighbors(rollout_rng))
solver = POMCPSolver(rollout_policy, 0.0, 100.0, 100, pomcp_rng, false, FullBeliefConverter(), 0)

POMCPSolver(DiscoveryHeuristic(ProbeHighestDegree(true),GuessBasedOnNeighbors(MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,0]),Uint32[0x00000001]))),0.0,100.0,100,MersenneTwister(DSFMT_state(Int32[1749029653,1072851681,1610647787,1072862326,1841712345,1073426746,-198061126,1073322060,-156153802,1073567984  …  1977574422,1073209915,278919868,1072835605,1290372147,18858467,1815133874,-1716870370,382,0]),Uint32[0x00000001]),false,FullBeliefConverter(),0)

In [29]:
@time est_rew(solver, nodes, comms, probes, p_intra, p_inter, N)

elapsed time: 61.699647723 seconds (2368848 bytes allocated, 0.08% gc time)


900.0